In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/variables/docker_base_image_python_version). Server response: 
HTTP code: 401, HTTP reason: Unauthorized, body: b'{"errorCode":200003,"errorMsg":"Invalidated Api key."}', error code: 200003, error msg: Invalidated Api key., user msg: 

In [2]:
feature_view = fs.get_feature_view(
    name='bitcoin_price_movement_training_fv',
    version=2
)

In [3]:
bitcoin_fg = fs.get_feature_group(
    name='bitcoin_price_movement',
    version=2
)

In [4]:
data = bitcoin_fg.select_all()
version = 2
feature_view = fs.get_or_create_feature_view(
    name='bitcoin_price_movement_training_fv',
    version=version,
    query=data
)

In [5]:
df = feature_view.get_batch_data()

Finished: Reading data from Hopsworks, using ArrowFlight (1.27s) 


In [3]:

import datetime as dt
from features import feature_engineering

In [7]:
start_date = dt.date(2016,1,1)
end_date = dt.date.today() - dt.timedelta(days=1)
df = feature_engineering.prepare_data(start_date=start_date, end_date=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2024-08-12 19:36:03,606 WARNING: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2024-08-12 19:36:04,034 WARNING: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

2024-08-12 19:36:04,036 WARNING: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


2024-08-12 19:36:04,328 WARNING: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


In [8]:
df

,date,open,high,low,close,volume,ma7,ma21,bollinger_upper,bollinger_lower,lag7,volatility,close_usd_index,close_oil,close_gold,hash_rate,id
2,2016-01-23,382.4340,394.5430,381.9810,387.4910,56247400,392.8701,418.0857,475.7586,360.4128,387.5360,15.7785,99.5700,32.1900,1097.2000,1008642.2324,1
3,2016-01-24,388.1020,405.4850,387.5100,402.9710,54824800,395.8233,416.7981,474.5601,359.0361,382.2990,15.4002,99.5700,32.1900,1097.2000,1008642.2324,2
4,2016-01-25,402.3170,402.3170,388.5540,391.7260,59062400,396.4744,414.8283,473.0760,356.5807,387.1680,15.0660,99.3600,30.3400,1106.2000,1008642.2324,3
5,2016-01-26,392.0020,397.7660,390.5750,392.1530,58147000,398.1893,412.9328,471.4292,354.4364,380.1490,13.4999,99.3600,31.4500,1121.7000,1008642.2324,4
6,2016-01-27,392.4440,396.8430,391.7820,394.9720,47424400,394.5810,411.3074,469.8133,352.8014,420.2300,9.3710,98.9000,32.3000,1116.1000,918901.0182,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,2024-08-06,53991.3477,57059.9180,53973.2734,56034.3164,49300484106,60030.6657,64380.9678,72530.9288,56231.0068,66201.0156,4240.1581,102.9700,73.2000,2389.1001,644509048.8165,3119
3121,2024-08-07,56040.6328,57726.8828,54620.5078,55027.4609,41637562185,58660.4102,63948.0472,73064.9859,54831.1086,64619.2500,4056.0119,103.2000,75.2300,2390.5000,644509048.8165,3120
3122,2024-08-08,55030.0273,62673.7656,54766.7266,61710.1367,45298472567,58139.3583,63840.2411,73009.2798,54671.2024,65357.5000,3195.2243,103.2100,76.1900,2422.2000,581410260.8205,3121
3123,2024-08-09,61728.2070,61751.8633,59587.8594,60880.1133,33425553115,58062.9364,63562.6200,72719.7348,54405.5052,61415.0664,3109.0551,103.1400,76.8400,2432.1001,581410260.8205,3122


In [10]:
sorted_df = df.sort_values(by='id')

In [11]:
import numpy as np

high_prices = sorted_df.loc[:, 'high'].values
low_prices = sorted_df.loc[:, 'low'].values
mid_prices = (high_prices + low_prices) / 2.0

mid_price_changes = np.diff(mid_prices) / mid_prices[:-1] * 100
mid_price_changes = np.insert(mid_price_changes, 0, 0)

features = sorted_df[['volume', 'ma7', 'ma21', 'bollinger_upper', 'bollinger_lower', 'volatility', 'close_usd_index', 'close_oil', 'close_gold', 'hash_rate']].values
feature_changes = np.diff(features, axis=0) / features[:-1] * 100
feature_changes = np.insert(feature_changes, 0, 0, axis=0)

combined_features = np.column_stack((mid_price_changes.reshape(-1, 1), feature_changes))

In [12]:
sequence_length = 100
sequence_data = []
sequence_labels = []

for i in range(len(combined_features) - sequence_length):
    sequence_data.append(combined_features[i:i + sequence_length])
    # Labels based on whether the next mid_price_change is positive (1) or negative (0)
    sequence_labels.append(1 if mid_price_changes[i + sequence_length] > 0 else 0)

sequence_data = np.array(sequence_data)
sequence_labels = np.array(sequence_labels)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

split_index = int(len(sequence_data) * 0.8)
train_data = sequence_data[:split_index]
train_labels = sequence_labels[:split_index]
test_data = sequence_data[split_index:]
test_labels = sequence_labels[split_index:]

train_data = TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
test_data = TensorDataset(torch.from_numpy(test_data), torch.from_numpy(test_labels))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [14]:
# Creating LSTM model class, which defines the model's structure
# We added dropout layer too, to try to tackle overfitting problem
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(self.dropout(lstm_out[:, -1, :]))
        return out

# Setting the input size of the model to match the number of features
# Setting the number of neurons (hidden size) to 50
# Setting the size of the output to 1, indicating that we will predict a single value (increase or decrease of the price)
input_size = combined_features.shape[1]
hidden_size = 30
output_size = 1

model = LSTMModel(input_size, hidden_size, output_size)

# Hyperparameters
epochs = 50
learning_rate = 0.001

train_losses = []
test_losses = []

# Early stopping parameters, these are added because early stopping method can reduce the risk of overfitting
# Early stopping stops the training process when the model's performance doesn't improve on a validation set anymore
# The patience parameter tells us through how many epochs we wait for improvement. If no improvement can be seen after 10, the training stops
# We track the loss of the model and stop when we don't see improvement on it
patience = 10
best_loss = float('inf')
epochs_no_improve = 0

# We use Binary Cross Entropy Loss function and combine it with a sigmoid layer in one function, which is needed for the classification problem
# We use Adam optimiser to adjust the parameters of the model to minimise loss during training
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
import torch
model = mr.get_model(
    name="bitcoin_price_movement_prediction_model_lstm", 
    version=1
)

saved_model_dir = model.download()

lstm_model = LSTMModel(input_size, hidden_size, output_size)
lstm_model.load_state_dict(torch.load(saved_model_dir + "/bitcoin_price_movement_prediction_lstm.pth"))
lstm_model.eval()

NameError: name 'mr' is not defined

In [19]:
import os
#import the model from the local directory
lstm_model = LSTMModel(input_size, hidden_size, output_size)

saved_model_dir = './lstm_model'

lstm_model.load_state_dict(torch.load(os.path.join(saved_model_dir, "best_model.pth")))

lstm_model.eval()

2024-08-12 20:04:01,996 WARNING: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


LSTMModel(
  (lstm): LSTM(11, 30, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [20]:
def predict(model, data_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for data, label in data_loader:
            data = data.float()
            output = model(data)
            predicted = torch.sigmoid(output).squeeze().tolist()
            predictions.extend(predicted)
            true_labels.extend(label.squeeze().tolist())

    predictions = [1 if p >= 0.5 else 0 for p in predictions]
    true_labels = [1 if a >= 0.5 else 0 for a in true_labels]

    return predictions, true_labels

In [21]:
predictions, labels = predict(lstm_model, test_loader)

predictions = np.array(predictions)
true_labels = np.array(labels)

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.56      0.48      0.52       266
           1       0.64      0.71      0.67       339

    accuracy                           0.61       605
   macro avg       0.60      0.59      0.59       605
weighted avg       0.60      0.61      0.60       605


In [25]:
# This function creates forecasted values into the future, based on the latest sequence
# It outputs a prediction whether the price would increase or decrease in the given time steps ahead in the future
def forecast(model, data, sequence_length, steps_ahead):
    model.eval()

    data = np.array(data)
    
    current_sequence = data[-sequence_length:].reshape(1, sequence_length, -1)
    predictions = []

    with torch.no_grad():
        for _ in range(steps_ahead):
            input_seq = torch.tensor(current_sequence, dtype=torch.float32)
            
            output = model(input_seq)
            predicted_value = torch.sigmoid(output).item()
            
            predicted_class = 1 if predicted_value >= 0.5 else 0
            
            predictions.append(predicted_class)
            
            new_sequence = np.append(current_sequence[0, 1:, :], [[predicted_class] * current_sequence.shape[2]], axis=0)
            current_sequence = new_sequence.reshape(1, sequence_length, -1)
    
    return predictions

# We use the forecast function on the combined dataset, for 2 days ahead in the future and print the predictions
all_data = np.concatenate((train_data.tensors[0].numpy(), test_data.tensors[0].numpy())).reshape(-1, input_size)
steps_ahead = 2
future_predictions = forecast(lstm_model, all_data, sequence_length, steps_ahead)
movement_interpretation = ['Increase' if pred == 1 else 'Decrease' for pred in future_predictions]

print('Future Predictions for next 2 days:', movement_interpretation)

Future Predictions for next 2 days: ['Decrease', 'Decrease']
